In [1]:
from toolbox import ToolBox
from tqdm import tqdm

from models import *

tqdm.pandas()

In [2]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
tb = ToolBox()

In [30]:
df = tb.load_data_sql(table='user_reviews_clean', use_cache=True)
df = df[~df['review'].isnull()]
df.head()

Loaded from cache


,id,date,grade,game,review,sentiment
0,128002,2013-01-19,30,007-legends-pc,"Oh, my dear God! What is it with developers th...",-0.9191
1,128003,2014-06-23,30,007-legends-pc,Seriously if you want to have a HUGE laugh wit...,0.9424
2,128004,2014-12-11,0,007-legends-pc,godawful port of the xbox 360 version and ps3....,-0.3869
3,128005,2012-11-03,0,007-legends-pc,Agree. Worst game ever. Its a full copy of Cal...,0.5496
4,128006,2012-11-04,0,007-legends-pc,DO NOT BUY THIS GAME for the PC (or for any ot...,-0.9612


In [31]:
min_dates = pd.DataFrame(df.groupby('game', as_index=False)['date'].min())

In [32]:
min_dates.rename(columns={'date':'release_date'}, inplace=True)
min_dates.head()

,game,release_date
0,007-legends-pc,2012-11-03
1,007-legends-playstation-3,2012-10-16
2,007-legends-wii-u,2013-01-04
3,007-legends-xbox-360,2012-10-16
4,007-quantum-of-solace-pc,2011-06-22


In [33]:
df = pd.merge(df, min_dates, on='game')
df.head()

,id,date,grade,game,review,sentiment,release_date
0,128002,2013-01-19,30,007-legends-pc,"Oh, my dear God! What is it with developers th...",-0.9191,2012-11-03
1,128003,2014-06-23,30,007-legends-pc,Seriously if you want to have a HUGE laugh wit...,0.9424,2012-11-03
2,128004,2014-12-11,0,007-legends-pc,godawful port of the xbox 360 version and ps3....,-0.3869,2012-11-03
3,128005,2012-11-03,0,007-legends-pc,Agree. Worst game ever. Its a full copy of Cal...,0.5496,2012-11-03
4,128006,2012-11-04,0,007-legends-pc,DO NOT BUY THIS GAME for the PC (or for any ot...,-0.9612,2012-11-03


In [34]:
def difference_weeks(date, release_date):
    date_dif=date-release_date
    weeks = math.floor(date_dif/np.timedelta64(1,'W'))
    return weeks

In [35]:
df['week'] = df.apply(lambda row: difference_weeks(row['date'], row['release_date']), axis=1)

In [58]:
df['period'] = df['week'].apply(lambda week: week//2)

In [59]:
df.head()

,id,date,grade,game,review,sentiment,release_date,week,period
0,128002,2013-01-19,30,007-legends-pc,"Oh, my dear God! What is it with developers th...",-0.9191,2012-11-03,11,5
1,128003,2014-06-23,30,007-legends-pc,Seriously if you want to have a HUGE laugh wit...,0.9424,2012-11-03,85,42
2,128004,2014-12-11,0,007-legends-pc,godawful port of the xbox 360 version and ps3....,-0.3869,2012-11-03,109,54
3,128005,2012-11-03,0,007-legends-pc,Agree. Worst game ever. Its a full copy of Cal...,0.5496,2012-11-03,0,0
4,128006,2012-11-04,0,007-legends-pc,DO NOT BUY THIS GAME for the PC (or for any ot...,-0.9612,2012-11-03,0,0


In [64]:
def get_reviews_per_period(n_periods, df):
    # define a dataframe 
    matrix = pd.DataFrame(np.zeros((len(df.groupby('game')), n_periods), dtype=np.int8), index=df.game.unique())
    
    # enter reviews
    for index, row in df[df['period']<n_periods].iterrows():
        matrix.loc[row['game'], row['period']] += 1
    
    return matrix

In [66]:
reviews_2weeks_games = get_reviews_per_period(5, df)

In [67]:
reviews_2weeks_games.head()

,0,1,2,3,4
007-legends-pc,7,1,0,1,0
007-legends-playstation-3,2,3,0,0,0
007-legends-wii-u,1,0,0,0,0
007-legends-xbox-360,8,1,2,0,1
007-quantum-of-solace-pc,2,0,1,0,0


In [68]:
reviews_2weeks_games.reset_index(inplace=True)
reviews_2weeks_games.rename(columns={'index': 'game'}, inplace=True)
reviews_2weeks_games.head()

,game,0,1,2,3,4
0,007-legends-pc,7,1,0,1,0
1,007-legends-playstation-3,2,3,0,0,0
2,007-legends-wii-u,1,0,0,0,0
3,007-legends-xbox-360,8,1,2,0,1
4,007-quantum-of-solace-pc,2,0,1,0,0


In [70]:
user_reviews = reviews_2weeks_games.melt(id_vars=['game'], var_name='period', value_name='nr_reviews')

In [71]:
user_reviews.sort_values(['game', 'period'], inplace=True)

In [73]:
user_reviews.head(10)

,game,period,nr_reviews
0,007-legends-pc,0,7
13068,007-legends-pc,1,1
26136,007-legends-pc,2,0
39204,007-legends-pc,3,1
52272,007-legends-pc,4,0
1,007-legends-playstation-3,0,2
13069,007-legends-playstation-3,1,3
26137,007-legends-playstation-3,2,0
39205,007-legends-playstation-3,3,0
52273,007-legends-playstation-3,4,0


In [74]:
user_reviews.shape

(65340, 3)

In [81]:
reviews_grouped = df.groupby(['game', 'period'], as_index=False).agg({'grade': 'mean', 'sentiment': 'std'})

In [82]:
reviews_grouped.rename(columns={'grade': 'user_avg_grade', 'sentiment': 'user_sentiment_var'}, inplace=True)

In [83]:
reviews_grouped.head()

,game,period,user_avg_grade,user_sentiment_var
0,007-legends-pc,0,1.428571,0.549131
1,007-legends-pc,1,100.000000,NaN
2,007-legends-pc,3,70.000000,NaN
3,007-legends-pc,5,30.000000,NaN
4,007-legends-pc,7,60.000000,NaN


In [84]:
user_reviews = pd.merge(user_reviews, reviews_grouped, how='left', on=['game', 'period'])

In [86]:
user_reviews.head(10)

,game,period,nr_reviews,user_avg_grade,user_sentiment_var
0,007-legends-pc,0,7,1.428571,0.549131
1,007-legends-pc,1,1,100.000000,NaN
2,007-legends-pc,2,0,NaN,NaN
3,007-legends-pc,3,1,70.000000,NaN
4,007-legends-pc,4,0,NaN,NaN
5,007-legends-playstation-3,0,2,25.000000,0.177060
6,007-legends-playstation-3,1,3,33.333333,0.820714
7,007-legends-playstation-3,2,0,NaN,NaN
8,007-legends-playstation-3,3,0,NaN,NaN
9,007-legends-playstation-3,4,0,NaN,NaN


In [87]:
user_reviews_lagged = user_reviews

In [88]:
user_reviews_lagged[['user_avg_grade', 'user_sentiment_var']] = user_reviews.groupby(['game'])['user_avg_grade', 'user_sentiment_var'].shift(1)

In [90]:
user_reviews_lagged.head(10)

,game,period,nr_reviews,user_avg_grade,user_sentiment_var
0,007-legends-pc,0,7,NaN,NaN
1,007-legends-pc,1,1,1.428571,0.549131
2,007-legends-pc,2,0,100.000000,NaN
3,007-legends-pc,3,1,NaN,NaN
4,007-legends-pc,4,0,70.000000,NaN
5,007-legends-playstation-3,0,2,NaN,NaN
6,007-legends-playstation-3,1,3,25.000000,0.177060
7,007-legends-playstation-3,2,0,33.333333,0.820714
8,007-legends-playstation-3,3,0,NaN,NaN
9,007-legends-playstation-3,4,0,NaN,NaN


In [91]:
critic_reviews = pd.read_pickle('critic_review_clean.pkl')

In [92]:
reviews_grouped = critic_reviews.groupby(['game'], as_index=False).agg({'grade': 'mean', 'sentiment': 'std', 
                                                                        'review': 'count'})
reviews_grouped.head()

,game,grade,sentiment,review
0,007-legends-pc,26.800000,0.351563,5
1,007-legends-playstation-3,42.318182,0.619410,22
2,007-legends-wii-u,40.666667,0.080168,3
3,007-legends-xbox-360,46.448980,0.669014,49
4,007-quantum-of-solace-ds,64.300000,0.411520,10


In [93]:
reviews_grouped.rename({'grade': 'critic_avg_grade', 'sentiment': 'critic_sentiment_var',
                       'review': 'critic_volume'}, inplace=True, axis=1)

In [94]:
reviews_grouped.head()

,game,critic_avg_grade,critic_sentiment_var,critic_volume
0,007-legends-pc,26.800000,0.351563,5
1,007-legends-playstation-3,42.318182,0.619410,22
2,007-legends-wii-u,40.666667,0.080168,3
3,007-legends-xbox-360,46.448980,0.669014,49
4,007-quantum-of-solace-ds,64.300000,0.411520,10


In [95]:
df = pd.merge(user_reviews_lagged, reviews_grouped, how='left', on='game')
df.head()

,game,period,nr_reviews,user_avg_grade,user_sentiment_var,critic_avg_grade,critic_sentiment_var,critic_volume
0,007-legends-pc,0,7,NaN,NaN,26.8,0.351563,5.0
1,007-legends-pc,1,1,1.428571,0.549131,26.8,0.351563,5.0
2,007-legends-pc,2,0,100.000000,NaN,26.8,0.351563,5.0
3,007-legends-pc,3,1,NaN,NaN,26.8,0.351563,5.0
4,007-legends-pc,4,0,70.000000,NaN,26.8,0.351563,5.0


In [96]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

In [97]:
data = df.set_index(['game', 'period'])
mod = PanelOLS.from_formula('''nr_reviews ~ 1 + user_sentiment_var*critic_sentiment_var + user_avg_grade + critic_avg_grade + 
                            critic_volume + TimeEffects''', data, )
res = mod.fit(cov_type='unadjusted')
res

/Users/myrthe/anaconda3/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Dep. Variable:,nr_reviews,R-squared:,0.0969
Estimator:,PanelOLS,R-squared (Between):,0.0993
No. Observations:,9049,R-squared (Within):,-0.0252
Date:,"Fri, Oct 25 2019",R-squared (Overall):,0.0922
Time:,12:04:54,Log-likelihood,-2.392e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,161.69
Entities:,4576,P-value,0.0000
Avg Obs:,1.9775,Distribution:,"F(6,9039)"
Min Obs:,1.0000,,
Max Obs:,4.0000,F-statistic (robust):,161.69


In [110]:
df_grouped = df.groupby(['game'], as_index=False).agg({'nr_reviews': 'min'})

In [111]:
df_grouped.rename(columns={'nr_reviews': 'min_reviews'}, inplace=True)

In [112]:
df_grouped.head()

,game,min_reviews
0,007-legends-pc,0
1,007-legends-playstation-3,0
2,007-legends-wii-u,0
3,007-legends-xbox-360,0
4,007-quantum-of-solace-pc,0


In [135]:
games_enough_reviews = list(df_grouped[df_grouped['min_reviews']>1]['game'])

In [136]:
len(games_enough_reviews)

483

In [137]:
df_games_enough_reviews = df[df['game'].isin(games_enough_reviews)]

In [138]:
df_games_enough_reviews.head()

,game,period,nr_reviews,user_avg_grade,user_sentiment_var,critic_avg_grade,critic_sentiment_var,critic_volume
55,1-2-switch-switch,0,36,NaN,NaN,58.190476,0.48737,65.0
56,1-2-switch-switch,1,12,63.888889,0.503946,58.190476,0.48737,65.0
57,1-2-switch-switch,2,2,53.333333,0.698784,58.190476,0.48737,65.0
58,1-2-switch-switch,3,2,0.000000,0.329582,58.190476,0.48737,65.0
59,1-2-switch-switch,4,2,55.000000,0.014991,58.190476,0.48737,65.0


In [139]:
df_games_enough_reviews.shape

(2415, 8)

In [140]:
data = df_games_enough_reviews.set_index(['game', 'period'])
mod = PanelOLS.from_formula('''nr_reviews ~ 1 + user_sentiment_var*critic_sentiment_var + user_avg_grade + critic_avg_grade + 
                            critic_volume + TimeEffects''', data, )
res = mod.fit(cov_type='unadjusted')
res

Dep. Variable:,nr_reviews,R-squared:,0.0275
Estimator:,PanelOLS,R-squared (Between):,0.0481
No. Observations:,1916,R-squared (Within):,-0.0102
Date:,"Fri, Oct 25 2019",R-squared (Overall):,0.0225
Time:,16:56:29,Log-likelihood,-5788.6
Cov. Estimator:,Unadjusted,,
,,F-statistic:,8.9725
Entities:,479,P-value,0.0000
Avg Obs:,4.0000,Distribution:,"F(6,1906)"
Min Obs:,4.0000,,
Max Obs:,4.0000,F-statistic (robust):,8.9725
